# Building LLM Agents with LLama3 and LangGraph

## What You'll Learn

In this notebook, you will learn how to build and deploy LLM (Large Language Model) agents using LLama3 and LangGraph. These agents will be capable of processing natural language inputs, generating comprehensive plans, and handling complex workflows. By the end of this notebook, you will understand how to:

1. Set up and configure a language model using LLama3.
2. Define and manage the state for agents in a workflow.
3. Implement and customize agent classes for specific tasks.
4. Construct and compile a workflow graph using LangGraph.
5. Execute the workflow and handle outputs effectively.

## Basic Concepts

Before diving into the implementation, it's essential to understand some basic concepts:

- **LLM (Large Language Model):** A machine learning model trained on vast amounts of text data to understand and generate human-like language. LLama3 is an example of such a model.

- **Agent:** A software component that interacts with the LLM to perform specific tasks, such as generating responses or processing information. In this notebook, we implement agents that can handle various aspects of a workflow.

- **State:** A shared data structure that stores the context and data required by agents. The state is critical for maintaining continuity and passing information between different parts of the workflow.

- **Workflow Graph:** A structured representation of the workflow, where nodes represent agents and edges define the flow of information and control. LangGraph is used to construct and manage these workflow graphs.

- **Prompt Engineering:** The process of crafting prompts that guide the LLM's responses. Proper prompt engineering is crucial for ensuring the model generates relevant and accurate outputs.

Understanding these concepts will provide a solid foundation as we proceed with the practical implementation of LLM agents and workflows in this notebook.

## 1. Setting Up the Environment

Before we dive into building our agent, we need to set up the necessary environment. This involves installing required packages and ensuring our Python environment is ready for development.

In [117]:
# Install termcolor for colored terminal outputs
%pip install termcolor langgraph

Note: you may need to restart the kernel to use updated packages.


In [118]:
# Import necessary libraries
from termcolor import colored
import json
import requests

## 2. Model Configuration

In this section, we set up the configuration for the Ollama model.

### Setting Up the Ollama Model

The `setup_ollama_model` function configures the model settings, including the endpoint, model name, system prompt, and other parameters. This setup is essential for initializing the model with the correct configuration, ensuring it can process queries and utilize the tools effectively.

In [119]:
def setup_ollama_model(model, temperature=0, stop=None):
    """
    Sets up the Ollama model configuration.

    Parameters:
    model (str): The name of the model to use.
    temperature (float): The temperature setting for the model.
    stop (str): The stop token for the model.

    Returns:
    dict: Configuration for the Ollama model.
    """
    return {
        "model_endpoint": "http://localhost:11434/api/generate",
        "model": model,
        "temperature": temperature,
        "headers": {"Content-Type": "application/json"},
        "stop": stop,
    }


# Example configuration
ollama_config = setup_ollama_model(model="llama3:instruct")

## 3: Defining the Agent Graph State

In this step, we define the structure of the state that our agents will use to store and communicate information. This state acts as a shared memory that different components of the system can access and modify. We use the `TypedDict` from the `typing` module to define the expected structure and types of data within the state. This helps ensure consistency and correctness when accessing or updating the state, making it easier to manage complex workflows and data dependencies.

The `AgentGraphState` class includes fields for the research question, responses from the planner agent, and any final outputs or end states. The `get_agent_graph`_state function is used to retrieve specific parts of the state based on a key, facilitating modular and reusable access to the state data.

In [120]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

# Define the state object for the agent graph
class AgentGraphState(TypedDict):
    """
    This class defines the structure of the agent graph state.
    
    Attributes:
    research_question (str): The main research question the agent is working on.
    planner_response (list): A list to store responses from the planner agent.
    end_chain (list): A list to store the final outputs or end states.
    """
    research_question: str
    planner_response: Annotated[list, add_messages]
    end_chain: Annotated[list, add_messages]

# Function to retrieve specific parts of the agent state
def get_agent_graph_state(state: AgentGraphState, state_key: str):
    """
    Retrieves specific parts of the agent state based on the provided key.
    
    Parameters:
    state (AgentGraphState): The current state of the agent.
    state_key (str): The key indicating which part of the state to retrieve.
    
    Returns:
    list or None: The requested state data or None if the key is not recognized.
    """
    if state_key == "planner_all":
        return state["planner_response"]
    elif state_key == "planner_latest":
        return state["planner_response"][-1] if state["planner_response"] else []
    else:
        return None

# Initial state setup
state = {
    "research_question": "",
    "planner_response": [],
    "end_chain": [],
}

## 4. Agent Class Definition

In this section, we define the `Agent` class, which serves as a base class for different types of agents in our system. An agent is a component that interacts with the language model to perform specific tasks, such as generating responses or processing information. The `Agent` class manages the configuration and state associated with the language model, allowing for easy setup and reuse of model configurations across different agents.

The class includes methods for initializing the agent with a specific model configuration and updating the agent's state. The state encapsulates the context or memory of the agent, enabling it to maintain continuity across interactions.

In [121]:
class Agent:
    def __init__(self, state: AgentGraphState, model_config: dict):
        """
        Initializes the agent with a state and model configuration.

        Parameters:
        state (AgentGraphState): The initial state of the agent, containing necessary context and data.
        model_config (dict): Configuration settings for the model, including endpoint, model name, temperature, etc.
        """
        self.state = state
        self.model_endpoint = model_config.get("model_endpoint")
        self.model_name = model_config.get("model")
        self.temperature = model_config.get(
            "temperature", 0
        )  # Default temperature is 0
        self.headers = model_config.get("headers", {"Content-Type": "application/json"})
        self.stop = model_config.get("stop")

    def update_state(self, key: str, value: any):
        """
        Updates the agent's state with a new key-value pair.

        Parameters:
        key (str): The key in the state dictionary to update.
        value (any): The new value to associate with the specified key.
        """
        if key in self.state:
            self.state[key] = value
        else:
            print(f"Warning: Attempting to update a non-existing state key '{key}'.")

## 5. Utility Functions

Utility functions are auxiliary functions that assist with various common tasks within the notebook. They help keep the codebase clean and modular by encapsulating frequently used logic in separate functions. In this case, we have two utility functions: `check_for_content` and `get_current_utc_datetime`.

- `check_for_content`: This function checks if a variable has a content attribute and returns its value if it exists. This is useful for handling different data types that may or may not have a content attribute.
- `get_current_utc_datetime`: This function returns the current date and time in UTC format. This can be useful for timestamping events or logging.

In [122]:
from datetime import datetime, timezone

# Check if an attribute of the state dict has content
def check_for_content(var):
    """
    Checks if the provided variable has a 'content' attribute and returns it.

    Parameters:
    var (Any): The variable to check.

    Returns:
    Any: The 'content' attribute if it exists, otherwise the original variable.
    """
    try:
        return var.content
    except AttributeError:
        return var


# Get the current date and time in UTC
def get_current_utc_datetime():
    """
    Returns the current date and time in UTC.

    Returns:
    str: The current date and time in UTC, formatted as 'YYYY-MM-DD HH:MM:SS UTC'.
    """
    now_utc = datetime.now(timezone.utc)
    return now_utc.strftime("%Y-%m-%d %H:%M:%S UTC")

## 6. PlannerAgent

In this section, we define the `planner_prompt_template` and the `PlannerAgent` class. The `planner_prompt_template` is a string template that sets the context and expectations for the planner agent's response. It guides the agent in generating a comprehensive plan to help answer a given research question. The planner's responsibilities include identifying the most relevant search term, outlining an overall strategy, and providing additional information or filters for a thorough search.

The `PlannerAgent` class extends the base `Agent` class and implements specific functionality for the planning task. It uses the provided template to format its requests to the language model and processes the model's responses. The class includes methods like `think`, which are used to generate and retrieve the agent's outputs based on the research question and any feedback received.

### Template Definition

In [123]:
# Template for guiding the planner agent's response
planner_prompt_template = """
You are a planner. Your responsibility is to create a comprehensive plan to help your team answer a research question. 
Questions may vary from simple to complex, multi-step queries. Your plan should provide appropriate guidance for your 
team to use an internet search engine effectively.

Focus on highlighting the most relevant search term to start with, as another team member will use your suggestions 
to search for relevant information.

If you receive feedback, you must adjust your plan accordingly. Here is the feedback received:
Feedback: {feedback}

Current date and time:
{datetime}

Your response must take the following JSON format:

    "search_term": "The most relevant search term to start with",
    "overall_strategy": "The overall strategy to guide the search process",
    "additional_information": "Any additional information to guide the search, including other search terms or filters"
"""

# JSON schema for the planner's response
planner_guided_json = {
    "type": "object",
    "properties": {
        "search_term": {
            "type": "string",
            "description": "The most relevant search term to start with",
        },
        "overall_strategy": {
            "type": "string",
            "description": "The overall strategy to guide the search process",
        },
        "additional_information": {
            "type": "string",
            "description": "Any additional information to guide the search, including other search terms or filters",
        },
    },
    "required": ["search_term", "overall_strategy", "additional_information"],
}

### PlannerAgent Class

In [124]:
from langchain_core.messages.human import HumanMessage

class PlannerAgent(Agent):
    
    def invoke(
        self,
        research_question: str,
        prompt: str = planner_prompt_template,
        feedback: any = None,
    ) -> dict:
        """
        Generates a response from the model based on the provided prompt.

        Parameters:
        research_question (str): The research question the planner agent needs to address.
        prompt (str): The template used to generate the system prompt.
        feedback (callable or str): Feedback received to adjust the planning process.

        Returns:
        dict: The updated state of the agent after processing the response.
        """
        feedback_value = feedback() if callable(feedback) else feedback
        feedback_value = check_for_content(feedback_value)

        planner_prompt = prompt.format(
            feedback=feedback_value, datetime=get_current_utc_datetime()
        )

        messages = [
            {"role": "system", "content": planner_prompt},
            {"role": "user", "content": f"Research question: {research_question}"},
        ]

        payload = {
            "model": self.model_name,
            "format": "json",
            "prompt": messages[1]["content"],
            "system": messages[0]["content"],
            "stream": False,
            "temperature": self.temperature,
        }

        try:
            response = requests.post(
                self.model_endpoint, headers=self.headers, data=json.dumps(payload)
            )
            response_json = response.json()
            response_content = json.loads(response_json.get("response", "{}"))
            response_formatted = HumanMessage(content=json.dumps(response_content))

            self.update_state("planner_response", response_formatted)
            print(colored(f"Planner 👩🏿‍💻: {response_formatted}", "cyan"))
            return self.state
        except requests.RequestException as e:
            print(f"Error in invoking model! {str(e)}")
            return {"error": str(e)}

## 7. End Node

The `EndNodeAgent` class is a specialized agent that marks the conclusion of the workflow in the agent graph. It extends the base `Agent` class and is primarily responsible for updating the state to indicate the end of the process. This agent is useful for workflows that require a clear termination point, ensuring that the system knows when all processing is complete.

In [125]:
class EndNodeAgent(Agent):
    def invoke(self) -> AgentGraphState:
        """
        Marks the end of the workflow by updating the state.

        This method updates the 'end_chain' key in the state to signify that
        the workflow has reached its conclusion. It can be used to perform any
        finalization tasks or simply to denote that the agent has completed its role.

        Returns:
        AgentGraphState: The updated state of the agent.
        """
        self.update_state("end_chain", "end_chain")
        return self.state

## 8. Creating and Compiling the Agent Graph

In this section, we define the structure and flow of the agent-based system using the `StateGraph` class from the `langgraph` library. The graph consists of nodes, each representing a specific agent, and edges, which define the flow or sequence of operations. This setup enables the modeling of complex workflows where different agents can interact and pass information.

- **`create_graph`:** This function initializes the `StateGraph` with a specific state structure (`AgentGraphState`). It then adds nodes for the `PlannerAgent` and `EndNodeAgent`, specifying the operations these agents should perform. The function sets the "planner" node as the entry point and the "end" node as the finish point, with an edge connecting them to define the workflow sequence.
- **`compile_workflow`:** This function compiles the defined graph into a workflow that can be executed. The compiled workflow manages the execution of the nodes in the defined order, handling the flow of data and control through the system.


In [126]:
from langgraph.graph import StateGraph, END

def create_graph() -> StateGraph:
    """
    Creates and configures the state graph for the agent workflow.

    This function initializes the graph, adds the necessary nodes (agents), and
    sets up the edges defining the flow of the workflow.

    Returns:
    StateGraph: The configured state graph for the workflow.
    """
    graph = StateGraph(AgentGraphState)
    graph.add_node(
        "planner",
        lambda state: PlannerAgent(
            state=state,
            model_config=ollama_config,
        ).invoke(
            research_question=state["research_question"],
            feedback=lambda: get_agent_graph_state(
                state=state, state_key="reviewer_latest"
            ),
            prompt=planner_prompt_template,
        ),
    )

    graph.add_node(
        "end",
        lambda state: EndNodeAgent(
            state=state,
            model_config=ollama_config,
        ).invoke(),
    )

    # Set the entry and finish points for the workflow
    graph.set_entry_point("planner")
    graph.set_finish_point("end")

    # Define the flow of the graph
    graph.add_edge("planner", "end")

    return graph


def compile_workflow(graph: StateGraph):
    """
    Compiles the state graph into an executable workflow.

    This function compiles the graph, enabling the defined nodes and edges to
    be executed in sequence as per the workflow's logic.

    Parameters:
    graph (StateGraph): The state graph defining the workflow.

    Returns:
    Any: The compiled workflow ready for execution.
    """
    workflow = graph.compile()
    return workflow

## Running the Workflow

In this final section, we execute the workflow defined in the agent graph. We start by creating the graph using the `create_graph` function and then compiling it into an executable workflow with `compile_workflow`. The workflow is then run with specific inputs and configurations.

- **`graph = create_graph()`:** This initializes the graph structure, including all nodes and edges as defined previously.
- **`workflow = compile_workflow(graph)`:** This compiles the graph into a runnable workflow, preparing it for execution.
- **`iterations = 10`:** This variable sets the recursion limit for the workflow, determining how many iterations the workflow should allow.
- **`verbose = True`:** If set to `True`, the system will print detailed information about each state change during the workflow execution.
- **`query = "Who's the president of the USA?"`:** The research question provided as input to the workflow, which the planner agent will process.
- **`dict_inputs = {"research_question": query}`:** A dictionary containing the initial inputs to the workflow, including the research question.
- **`limit = {"recursion_limit": iterations}`:** This sets the limit for the number of iterations, preventing infinite loops or excessive processing.

The loop iterates over the events generated by the workflow, printing the state at each step if `verbose` is enabled.

In [127]:
# Create the graph and compile the workflow
graph = create_graph()
workflow = compile_workflow(graph)
print("Graph and workflow created.")

# Define workflow parameters
iterations = 10
verbose = True
query = "Who's the president of the USA?"
dict_inputs = {"research_question": query}
limit = {"recursion_limit": iterations}

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, limit):
    if verbose:
        print("\nState Dictionary:", event)
    else:
        print("\n")

Graph and workflow created.
Planner 👩🏿‍💻: content='{"search_term": "president usa", "overall_strategy": "Use the internet search engine with the default settings and start with a broad search query. If no clear answer is found, consider using quotation marks to search for exact phrases and also try adding relevant keywords such as \'current\' or \'latest\' to narrow down the results.", "additional_information": "You can also use specific years to get information about past presidents, for example: \'president usa 2024\', \'president usa 1990s\'."}'

State Dictionary: {'planner': {'research_question': "Who's the president of the USA?", 'planner_response': HumanMessage(content='{"search_term": "president usa", "overall_strategy": "Use the internet search engine with the default settings and start with a broad search query. If no clear answer is found, consider using quotation marks to search for exact phrases and also try adding relevant keywords such as \'current\' or \'latest\' to narr